
# Predicción de Resultados de Carreras de F1 con Machine Learning 🏎️

Este notebook entrena un modelo para predecir la posición final de los pilotos en carreras de Fórmula 1 usando datos históricos.


In [ ]:

# Instalar paquetes necesarios (si no están instalados)
!pip install pandas numpy scikit-learn matplotlib seaborn xgboost


In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from xgboost import XGBClassifier
import warnings
warnings.filterwarnings('ignore')


In [ ]:

# Cargar datos de ejemplo (puedes reemplazar con dataset más actualizado si deseas)
url = "https://raw.githubusercontent.com/jakelever/f1-circuits/master/f1data/results.csv"
df = pd.read_csv(url)
df.head()


In [ ]:

# Usamos solo algunas columnas útiles y eliminamos nulos
df = df[['raceId', 'driverId', 'constructorId', 'grid', 'positionOrder']].dropna()

# Convertir a enteros
df = df.astype(int)

# Definir variables de entrada (X) y salida (y)
X = df[['raceId', 'driverId', 'constructorId', 'grid']]
y = df['positionOrder']

# Para simplificar, vamos a convertir las posiciones a una clasificación Top 3, Top 10, o fuera del Top 10
def categorize_position(pos):
    if pos <= 3:
        return "Top3"
    elif pos <= 10:
        return "Top10"
    else:
        return "Others"

y = y.apply(categorize_position)


In [ ]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:

model = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
model.fit(X_train, y_train)


In [ ]:

y_pred = model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt="d", cmap="Blues",
            xticklabels=np.unique(y_test), yticklabels=np.unique(y_test))
plt.title("Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.show()


In [ ]:

# Simular una predicción: piloto 1, equipo 9, desde la posición 3 en la parrilla de salida, en carrera 20
sample = pd.DataFrame([[20, 1, 9, 3]], columns=['raceId', 'driverId', 'constructorId', 'grid'])
print("Predicción:", model.predict(sample)[0])
